In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 47.3 MB 93 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595734 sha256=6013175dcc64e2837adea9afbed1943bfa8e8c0f38846c411f4011ffe1e1715f
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.4 MB/s 
     |████████████████████████████████| 769 kB 5.5 MB/s 
     |████████████████████████████████| 895 kB 28.0 MB/s 
     |████████████████████████████████| 3.0 MB 32.9 MB/s 


In [2]:
!pip install qiskit

     |████████████████████████████████| 6.5 MB 4.4 MB/s 
     |████████████████████████████████| 18.0 MB 73 kB/s 
     |████████████████████████████████| 238 kB 38.5 MB/s 
     |████████████████████████████████| 200 kB 17.2 MB/s 
     |████████████████████████████████| 54 kB 1.0 MB/s 
     |████████████████████████████████| 1.6 MB 30.0 MB/s 
     |████████████████████████████████| 943 kB 36.2 MB/s 
     |████████████████████████████████| 38.1 MB 1.6 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 37.5 MB 381 kB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
     |████████████████████████████████| 3.6 MB 34.5 MB/s 
     |████████████████████████████████| 113 kB 34.2 MB/s 
  Created wheel for qiskit: filename=qiskit-0.34.2-py3-none-any.whl size=11805 sha256=2eb8d6d8781cc39838d88e40c8a55a026d5aa8ce2791d4766ac83220e2422a78
  Stored in directory: /root/.cache/pip/wheels/62/77/65/cda6eedfdd2a525bd3f479a4386930ae3088a1eb01f8c94

In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-_jzmjfrp
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-_jzmjfrp
     |████████████████████████████████| 132 kB 5.0 MB/s 
     |████████████████████████████████| 4.5 MB 32.4 MB/s 
     |████████████████████████████████| 3.5 MB 32.6 MB/s 
     |████████████████████████████████| 596 kB 40.4 MB/s 
     |████████████████████████████████| 6.8 MB 30.7 MB/s 
     |████████████████████████████████| 67 kB 4.4 MB/s 
     |████████████████████████████████| 79 kB 6.3 MB/s 
     |████████████████████████████████| 8.6 MB 33.6 MB/s 
     |████████████████████████████████| 127 kB 41.2 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=c4f2d1c3d410aeebdd12d58f12fe8e0ffef3d13d3f9da0014a0785bd8aa65ea0
  Stored in directory: /tmp/pip-ephem-wheel-cache-s_48lu_9/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd05

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
# pytorch 에서 gpu 사용 선택 (0번 아이디를 가진 gpu를 사용)
device = torch.device("cuda:0")

# pytorch 에서 cpu 사용 선택
# device = torch.device("cpu")

In [9]:
# 사전 훈련된 kobert model 불러옴
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [11]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [112]:
# 구글 드라이브에서 학습시킬 데이터 셋 가져오기
# /content/drive/MyDrive/ == 기본 경로
import pandas as pd
sentiment_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/kobert/sentiDataset.xlsx')
sentiment_data2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/kobert/AIHUB_dataset_customizing.xlsx')

In [113]:
# column 지정
sentiment_data.columns = ["Sentence","Emotion"]

In [114]:
# non 있는 행 제거
sentiment_data = sentiment_data.dropna(axis=0)

In [117]:
# 데이터 셋 크기 확인
print(len(sentiment_data))
print(len(sentiment_data2))

502
38594


In [118]:
sentiment_data = pd.concat([sentiment_data,sentiment_data2])

In [119]:
print(len(sentiment_data))

39096


In [120]:
# 데이터 셋 같은 경우, 0~2 까지 3개의 분류로 구성되어 있음 
# 해당 데이터 셋의 공포, 놀람 등 항목을 사용하기 편하도록 숫자로 변경
sentiment_data.loc[(sentiment_data['Emotion'] == "긍정"), 'Emotion'] = 0  #긍정 => 0
sentiment_data.loc[(sentiment_data['Emotion'] == "부정"), 'Emotion'] = 1  #부정 => 1
sentiment_data.loc[(sentiment_data['Emotion'] == "중립"), 'Emotion'] = 2  #중립 => 2

In [121]:
# 학습 데이터 셋 구성
# 가져온 데이터 셋 파일에서 sentence와 emotion 항목을 q와 label로 각각 구성하여 학습을 위한 데이터 셋으로 구성 
data_list = []
for q, label in zip(sentiment_data['Sentence'], sentiment_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [122]:
# 데이터 셋의 형태 확인
print(len(data_list))
print(data_list[0])
print(data_list[-1])

39096
['민주당 진즉 제대로 했으면 이런 힘든 대선 경쟁을 안하잖아', '1']
['와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요', '1']


In [123]:
# sklearn.model_selection 패키지를 통하여 학습 데이터와 테스트 데이터로 분할
from sklearn.model_selection import train_test_split

# 75%는 학습 데이터셋, 25%는 테스트 데이터셋으로 구성
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [124]:
# 학습 데이터셋과 테스트 데이터셋의 분할 확인 및 형태 확인
print(len(dataset_train))
print(len(dataset_test))
print(dataset_train[0])
print(dataset_test[0])

29322
9774
['지금 내모습은 컴퓨터앞에서 하루종일 취업사이트만 검색중...', '1']
['친일우빨알바들 전라도,빨갱이 없으면 어떻게 사나', '1']


In [125]:
# 학습에 사용할 데이터 셋 클래스 선언
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        # transform하여 토큰화 하는 과정에서 
        # transform(i[인덱스]) == transform("아아아아 아아아 아아") 같은 경우 토큰화가 정상적으로 진행 안되는 오류 
        # transform([i[인덱스]]) == transform(["아아아아 아아아 아아"]) 형식으로 transform 진행
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    # 전체 데이터 셋에서 해당하는 인덱스의 sentences와 labels만 뽑는 함수
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    # 데이터 셋의 사이즈 return
    def __len__(self):
        return (len(self.labels))

In [126]:
# 모델 파라미터 설정
# 토큰의 최대 길이라고 생각
max_len = 64
# 몇 개의 샘플들을 예측해보고 가중치를 업데이트 할 지 설정
# 아래와 같이 배치 사이즈가 64인 경우 데이터 64개 마다 예측한 것을 실제 값과 비교한다
batch_size = 64
warmup_ratio = 0.1
# epoch 횟수는 모델이 전체 데이터셋을 훈련시킬 횟수를 의미한다.
num_epochs = 10
max_grad_norm = 1
log_interval = 200
# learning_rate 값이 너무 크면 원하는 값에 도달하기 힘들고, 너무 작으면 학습기간이 오래 걸린다.
learning_rate =  5e-5

In [127]:
# kobert에서 vocab을 통해서 토큰화 진행
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [128]:
# 학습 데이터셋과 테스트 데이터셋 토큰화 진행
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [129]:
# 토큰화 확인 (마지막에 호출된 데이터셋이 출력됨)
data_train[0]
# data_test[0]

(array([   2, 4299, 1434, 6213, 6699, 7086, 4660, 6832, 6903, 4937, 7268,
        7126, 4604, 6510, 6150,  898, 7295,  517,   54,  517,   54,  517,
          54,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(24, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [130]:
# DataLoader를 통해서 전체 데이터셋이 batch_size로 분할
# num_workers의 경우 높을수록 load 속도 상승
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [131]:
# 분류에 사용할 모델 클래스 선언
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 # num_classes는 카테고리의 개수를 의미한다. (현재 데이터셋의 경우 3개의 분류로 데이터셋이 구성되어 있음)
                 num_classes=3,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [132]:
# 사전 훈련된 kobert model 불러옴
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [133]:
# to(Device)는 위의 torch.device("cuda:0")를 의미하며 
# GPU에서 학습된 모델을 GPU로 불러올 때 사용한다.
# 또한 GPU로 학습된 모델에 데이터를 제공할 때도 to(Device)를 붙여줘야 함.
# bert모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [134]:
# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [135]:
# optimizer = 전체 데이터 셋의 실제 결과와 모델이 예측한 값 간의 차이가 효율적으로 좁혀질 수 있도록 최적화해주는 역할
# transform에서 제공하는 AdamW optimizer 사용
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
# 예측값과 실제값의 오차인 손실함수는 pytorch에서 제공하는 다중분류를 위한 대표적인 손실함수인 torch.nn.CrossEntropyLoss 사용
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [136]:
# 총 학습 수는 학습데이터의 크기 * 에폭수
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [137]:
# 시간이 지남에 따라 학습률을 조금씩 감소시키는 scheduler
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [138]:
# 정확도 측정 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [139]:
# 모델 학습 
# 에폭수 만큼 반복
for e in range(num_epochs):
    # 정확도 초기화
    train_acc = 0.0
    test_acc = 0.0
    # 학습 모드
    model.train()

    # 배치만큼 학습 데이터셋 가져옴
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        # 그래디언트 = 순간변화율
        # 그래디언트 초기화
        optimizer.zero_grad()

        # 배치에서 데이터 추출
        # 모델에 제공하는 데이터이기때문에 to(device)
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        # output
        out = model(token_ids, valid_length, segment_ids)

        # 손실 계산 (실제값과 예측값 간의 오차)
        loss = loss_fn(out, label)

        # 그래디언트에 값 더하기
        loss.backward()

        # 학습의 안정화를 위해 그래디언트 클리핑(자르기) => 기울기가 너무 커지는 것을 방지
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        # 가중치 파라미터 업데이트
        optimizer.step()

        # 학습률 감소 스케줄러 
        scheduler.step()

        # 정확도
        train_acc += calc_accuracy(out, label)

        # batch 200 간격으로 학습상황 출력
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    # 평가 모드
    model.eval()

    # 테스트 데이터셋을 통한 평가
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/459 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.0462727546691895 train acc 0.4375
epoch 1 batch id 201 loss 0.7998729348182678 train acc 0.5786691542288557
epoch 1 batch id 401 loss 0.6784334182739258 train acc 0.6417160224438903
epoch 1 train acc 0.6507421023965142


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 9 test acc 0.7348989414606423


  0%|          | 0/459 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.12435292452573776 train acc 0.953125
epoch 10 batch id 201 loss 0.008842183277010918 train acc 0.9902052238805971
epoch 10 batch id 401 loss 0.003580370219424367 train acc 0.9913887157107232
epoch 10 train acc 0.9913875272331155


  0%|          | 0/153 [00:00<?, ?it/s]

epoch 10 test acc 0.7382690394998579


In [140]:
# 구글 드라이브 기본 경로 = '/content/drive/MyDrive'
import os
# 해당 디렉토리로 이동
os.chdir('/content/drive/MyDrive/Colab Notebooks/kobert')
# 현재 경로
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/kobert'

In [141]:
path = '/content/drive/MyDrive/Colab Notebooks/kobert/'
# 지정한 path에 전체 모델 저장
torch.save(model, path + 'sentiment_model.pt') 

# dict 저장
torch.save(model.state_dict(), 'sentiment_model_state_dict.pt')

# 필요한 값 지정해서 저장 가능
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, 'sentiment_model_all.tar')